<a href="https://colab.research.google.com/github/ivoryRabbit/kakao_arena/blob/master/5_ColdStart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cold start
- 플레이리스트의 제목, 곡, 태그에 대해 데이터 희소성(sparsity)가 발생하는 경우.
- 베스트셀러(bestseller) 기반의 모델을 만들어 적용

# 고려 대상
- 제목이 없는 경우
- 곡이 없는 경우
- 태그가 없는 경우
- 제목과 태그가 없는 경우
- 제목과 곡이 없는 경우
- 태그와 곡이 없는 경우
- 모두 없는 경우

# 공통 사항
- 플레이리스트 생성 날짜 이후의 곡들은 제거
  - 점수 상향 보장 (0.05 정도)
- 추천 목록의 순서
  - 평가지표가 __nDCG__이기 때문에 순서가 매우 중요!!!
  - 추천시 아이템의 가중치(rating)를 계산해야함
  - 앙상블 유형에 따라 모델 별 rating 측정이 요구됨
    - ex) ALS 50곡, Embedding 50곡을 각각 추천하는 경우에는 sorting이 중요해지기 때문에 사전에 rating을 계산하여야 한다. 그러나 두 모델의 rating의 scale이 다르기 때문에 쉽지 않아보임
- 동명의 플레이리스트 존재
  - id는 다르더라도 플레이리스트 제목이 같은 경우 존재
  - 일일이 찾아서 채워주면 될듯

# 태그 추천
1. 태그가 있으면 tag에 대한 CF
2. 태그가 없고 곡만 있으면 CF처럼 곡으로 태그 유추
3. 태그가 없고 곡도 없으면 제목으로 NLP
4. 아무것도 없으면 bestseller 기반으로

# 곡 추천(태그 추천 후)
1. 곡이 있으면 곡에 대한 CF
2. 곡이 없고 태그만 있으면 태그 별 best seller
3. 곡이 없고 태그도 없으면 bestseller기반

# 아이디어
1. sentencepiece 사용
2. 앨범이름, 곡이름, tag PCA해서 회귀?
3. 플레이리스트 업데이트 시간 기준으로 쪼개서 과거, 현재 비교
  - 낮은 행열 밀도로 인한 데이터 희소성 문제, 메모리 문제 해결
4. 플레이리스트 클러스터링

# CF 전략
1. 곡x 태그x : 베스트셀러기반
2. 곡o 태그x : 곡에 대한 ALS
3. 곡x 태그o : 태그에 대한 ALS
4. 곡o 태그o : NLP - embedding

# ALS 추가
- 현재 CF 알고리즘의 measure은 inner product지만 cosine similarlity 혹은 correlation 사용 가능
- 치환수식: $x \cdot y - n \cdot \mu(x) \cdot \mu(y)$
- CF와 CBF 두 모델을 성능이 좋은 쪽에 가중치를 두고 결합
  - ex) [0.7 x CF + 0.3 x CBF] $\cdot$ [populartion]
- 혹은 CF를 적용하면서 유사도가 높은 다른 user 정보가 없는 경우, 이를 수치로 기록해두고 나중에 CBF를 적용
- CF로 못 맞춘 데이터에 대한 분석도 필요
- {제목 token: tag} mapping을 이용하여 TF-IDF처럼 키워드 분석 진행
  - ex) 우리의 만남 -> [우리, 만남]
  - 우리: 사랑, 이별
  - 만남: 사랑, 연인, 감성
  - 우리의 만남 -> {사랑:2, 연인:1, 감성:1, 이별:1}

# Embedding
1. 분석 요소
  - 태그, 플레이리스트 제목, 곡 앨범 제목, 곡 이름, 아티스트, 장르
2. 분석 목표
  1. embedding 생성
  2. 키워드 분석
3. 분석 요소에 대해 tokenizing
  - embedding에 필요한 요소들의 차원 축소 및 상관 형성
  - EDA 및 전처리에도 사용
4. tokenizing 결과에 대해 word2vec 모델 이용
  - tokenizing 없이도 높은 성능을 보임
  - tokenizing 한다면 성능이 높아질 것으로 보임
  - cosine similarity 이용에 목적을 둠(= CF)
  - gensim 외에 직접 tensorflow로 모델 개발 가능
5. 추후 사항
  - embedding + meta data를 이용하여 autoencoder 사용가능
  - GCN을 이용하면 CF 효과 증대될 것
  - embedding을 이용하여 CBF 적용

# Hybrid
- 추천시스템에서 앙상블(Ensemble)의 개념에 해당
- 전략
  1. 여러 모델 학습 후, 각 모델의 추천 점수(rating)를 가중평균
    - 모델의 성격이 다르면 가중치를 설정하기 어렵다
    - 방법을 찾아봐야함
  2. 모델의 추천 결과를 단순 혼합
    - nDCG는 순서를 중시함으로 사실상 성능 하락을 야기
    - 현재 우리상황에서 거의 사용 불가
  3. 모델을 합성(composition)하여 사용
    - 태그와 곡을 추천하는 모델을 각각 만들어 서로 재귀적으로 학습시키면 성능이 좋아질 것잉라 생각됨
    - 다만 작업환경이 열악하여 가능할지 모르겠음
  4. 모델의 추천결과에 따라 데이터를 분류(잘 맞춘 데이터 vs 못 맞춘 데이터)하여 새로운 모델에 적용
    - 가장 가능성 있지않나 싶음
  5. 애초에 데이터를 클러스터링하여 서로다른 모델을 적용
    - 비지도로 클러스터링하기엔 위험부담이 있어, 4번 전략의 도움을 받아야함

## ColdStart 점수

- 07/13 BestSeller
    - song: 0.022775
    - tag: 0.161075

- 07/18 Charater-Level Skip-Gram
    - song: 0.035285
    - tag: 0.199979


In [ ]:
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/notebooks'
try: os.symlink('/content/drive/My Drive/Colab Notebooks/my_env', my_path)
except: print('Already does')
sys.path.insert(0, my_path)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import re, glob
import json
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm

from gensim.models import Word2Vec
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors

In [ ]:
display(glob.glob('drive/My Drive/kakao_arena/data/*'))
base_path = 'drive/My Drive/kakao_arena/'

['drive/My Drive/kakao_arena/data/val.json',
 'drive/My Drive/kakao_arena/data/test.json',
 'drive/My Drive/kakao_arena/data/genre_gn_all.json',
 'drive/My Drive/kakao_arena/data/train.json',
 'drive/My Drive/kakao_arena/data/song_meta.json',
 'drive/My Drive/kakao_arena/data/val_v1.json',
 'drive/My Drive/kakao_arena/data/test_v1.json',
 'drive/My Drive/kakao_arena/data/train_v1.json',
 'drive/My Drive/kakao_arena/data/genre_v1.json',
 'drive/My Drive/kakao_arena/data/song_meta_v1.json',
 'drive/My Drive/kakao_arena/data/test_v2.json',
 'drive/My Drive/kakao_arena/data/val_v2.json',
 'drive/My Drive/kakao_arena/data/train_v2.json',
 'drive/My Drive/kakao_arena/data/song_meta_v2.json']

In [ ]:
train = pd.read_json(base_path + 'data/train_v2.json')
valid = pd.read_json(base_path + 'data/val_v2.json')
test = pd.read_json(base_path + 'data/test_v2.json')

In [ ]:
song_meta = pd.read_json(base_path + 'data/song_meta_v2.json')
song_meta.head(3)

,song_gnr_dtl_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gnr_basket,artist_name_basket,songs,song_gnr_keywords,like_cnt,popular,song_name_basket,album_name_basket
0,[GN0901],20140512,불후의 명곡 7080 추억의 얄개시대 팝송베스트,2255639,[],feelings,[GN0900],[],0,"[국외, 팝]",0.058775,0.142951,[feeling],"[불후, 명곡, 추억, 시대, 팝송, 베스트]"
1,"[GN1601, GN1606]",20080421,bach partitas nos 2 3 4,376431,[29966],bach partita no 4 in d major bwv 828 ii allemande,[GN1600],[murray perahia],1,"[독주곡, 클래식]",0.000000,0.000000,"[bach, part, major, bwv, man]","[bach, part]"
2,[GN0901],20180518,hit,4698747,[3361],solsbury hill remastered 2002,[GN0900],[peter gabriel],2,"[국외, 팝]",0.000000,0.000000,[remaster],[hit]


## 1. BestSeller Recommendation

In [ ]:
tag_dic = {str(train.at[idx, 'id']): train.at[idx, 'tags'] for idx in tqdm(train.index)}
song_dic = {str(train.at[idx, 'id']): train.at[idx, 'songs'] for idx in tqdm(train.index)}
date_dic = {idx: song_meta.at[idx, 'issue_date'] for idx in tqdm(song_meta.index)}

In [ ]:
song_rank = train.songs.explode().value_counts().index.to_list()
tag_rank = train.tags.explode().value_counts().index.to_list()

In [ ]:
def remove_seen(seen, items):
    seen = set(seen)
    return [i for i in items if i not in seen]

def remove_pass(date, songs):
    return [s for s in songs if date_dic[s] <= date]

In [ ]:
def BestSeller(df):
    res = []
    song_cand = song_rank[:2000]
    tag_cand = tag_rank[:20]
    for idx in tqdm(df.index):
        songs, tags, date = df.at[idx, 'songs'], df.at[idx, 'tags'], df.at[idx, 'updt_date']
        res.append({
            'id': df.at[idx, 'id'],
            'songs': remove_seen(songs, remove_pass(date, song_cand)[:200])[:100],
            'tags': remove_seen(tags, tag_cand)[:10]
        })
    res = pd.DataFrame(res)
    return res

In [ ]:
valid_cold = BestSeller(valid)

In [ ]:
valid_cold.to_json(base_path + 'submissions/valid_cold_1.json', orient = 'records', force_ascii = False)

In [ ]:
test_cold = BestSeller(test)

In [ ]:
test_cold.to_json(base_path + 'submissions/test_cold_1.json', orient = 'records', force_ascii = False)

## 2. Title-based Recommendation

- Use character-level skip-gram

In [ ]:
valid_cold = pd.read_json(base_path + 'submissions/valid_cold_1.json')
valid_cold.head(3)

,id,songs,tags
0,118598,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
1,131447,"[116573, 366786, 654757, 133143, 675115, 61093...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
2,51464,"[654757, 253755, 117595, 645489, 13198, 88503,...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"


In [ ]:
test_cold = pd.read_json(base_path + 'submissions/test_cold_1.json')
test_cold.head(3)

,id,songs,tags
0,70107,"[116573, 366786, 654757, 133143, 610933, 13281...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
1,7461,"[144663, 116573, 357367, 366786, 654757, 13314...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 드라이브, 힐링, 사랑, 새벽, 밤]"
2,90348,"[116573, 366786, 654757, 133143, 675115, 61093...","[기분전환, 감성, 휴식, 발라드, 잔잔한, 힐링, 사랑, 새벽, 밤, 카페]"


In [ ]:
def Embedding(Series, dim):
    items = Series.tolist()
    items = [i for i in items if i != []]
    W2V = Word2Vec(items,
                   window = max([len(i) for i in items]), 
                   size = dim, 
                   min_count = 3, 
                   workers = 4,
                   negative = 5,
                   sg = 1)
    print('finish...')
    return W2V

def Embedding_Mapper(Series, model):
    ID = []
    vec = []
    for idx in tqdm(Series.index):
        N = 0
        temp_vec = 0
        for w in Series[idx]:
            try: temp_vec += model.wv.get_vector(w)
            except: pass
            else: N += 1
        if type(temp_vec) == int:
            continue
        ID.append(str(idx))
        vec.append(temp_vec / N)
    vec = np.array(vec)

    WEKV = WordEmbeddingsKeyedVectors(vec.shape[1])
    WEKV.add(ID, vec)
    return WEKV

In [ ]:
def make_tokens(df):
    up_df = df.set_index('id')
    title_char = up_df.plylst_title.apply(list)
    title_char = title_char.apply(lambda x: [y for y in x if y != ' '])
    return title_char + up_df.title_basket

In [ ]:
train_title_char, valid_title_char, test_title_char = map(make_tokens, (train, valid, test))

id
61281                     [여, 행, 같, 은, 음, 악, 여행, 음악]
10532                            [요, 즘, 너, 말, 야, 요즘]
76951    [편, 하, 게, 잔, 잔, 하, 게, 들, 을, 수, 있, 는, 곡, 잔잔]
dtype: object

In [ ]:
# train
%%time
char2vec = Embedding(train_title_char, 256)

finish...
CPU times: user 2min 41s, sys: 277 ms, total: 2min 41s
Wall time: 1min 22s


In [ ]:
train_mapped, valid_mapped, test_mapped = map(lambda x: Embedding_Mapper(x, char2vec), (train_title_char, valid_title_char, test_title_char))

In [ ]:
def CLSG(df, base_df, df_mapped): # Character-Level Skip-Gram Recommendation for ColdStart
    res = []
    for idx in tqdm(df.index):
        songs, tags, date, ID = df.at[idx, 'songs'], df.at[idx, 'tags'], df.at[idx, 'updt_date'], df.at[idx, 'id']
        try:
            most_id = [x[0] for x in train_mapped.similar_by_vector(df_mapped.get_vector(str(ID)), topn = 200)]
            get_songs, get_tags = [], []
            for i in most_id:
                get_songs += song_dic[i]
                get_tags += tag_dic[i]
            get_songs = list(pd.value_counts(get_songs)[: 200].index)
            get_tags = list(pd.value_counts(get_tags)[: 20].index)

            get_songs = remove_seen(songs, remove_pass(date, get_songs)[:200])[:100]
            get_tags = remove_seen(tags, get_tags)[: 10]

            if len(get_songs) < 100:
                get_songs += remove_seen(get_songs, base_df.at[idx, 'songs'])[: 100 - len(get_songs)]
            if len(get_tags) < 10:
                get_tags += remove_seen(get_tags, base_df.at[idx, 'tags'])[: 10 - len(get_tags)]

            res.append({
                'id': ID,
                'songs': get_songs,
                'tags': get_tags
            })
        except:
            res.append({
                'id': ID,
                'songs': base_df.at[idx, 'songs'],
                'tags': base_df.at[idx, 'tags']
            })
    res = pd.DataFrame(res)
    return res

In [ ]:
valid_cold_2 = CLSG(valid, valid_cold, valid_mapped)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
valid_cold_2.to_json('drive/My Drive/kakao_arena/submissions/valid_cold_2.json', orient = 'records', force_ascii = False)

In [ ]:
test_cold_2 = CLSG(test, test_cold, test_mapped)

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [ ]:
test_cold_2.to_json('drive/My Drive/kakao_arena/submissions/test_cold_2.json', orient = 'records', force_ascii = False)